# Transformers and Transfer Learning

이제 자연어 처리 분야를 소개받았으므로 이해해야 할 중요한 사항이 있습니다. 실제로 최첨단 기술을 시작하는 곳에서 시작하는 것은 그리 긴 여정이 아닙니다.

결국 우리는 기본으로 돌아가서 기본에 대해 논의하고 모든 세부 사항을 이해하게 될 것입니다. 그러나 우리는 약속의 영역에 도달하기 위한 길고 힘든 여정을 시작하기 전에 약속의 영역을 보여줄 것입니다.
    
실제 세계에서 딥 러닝을 작동시키려는 경우 구현해야 할 가장 중요한 아이디어 중 하나는 전이 학습입니다. 이는 다른 데이터 세트에서 이미 훈련된 모델을 가져와 새 데이터 세트에 맞게 미세 조정하는 프로세스입니다. 예를 들어, Hemingway 스타일의 매력적인 단편 소설을 생성하기 위해 언어 모델을 훈련하는 경우 Hemingway의 텍스트 샘플에 대해서만 훈련하는 대신 다양한 책에 대해 훈련된 모델을 미세 조정할 수 있습니다. 많지 않을 수 있습니다.

.Who's That Pokémon? Language Models
> Tip: A language model is a function that takes in a sequence of words and returns a probility distribution over all the possible next words in that sequence. This task is considered one of the most important in NLP because, as the reasoning goes, to predict the next word in a sentence, you **must** have a good understanding of the language. Language models learn the features and characteristics of language in order to guess what the next word should be after any given prior phrase or sentence. Language models are the backbone of NLP today because they do not require explicit annotations (labels) and can be trained on massive corpuses without any material data preparation. Once they learn the properties of language well, language models can be fine-tuned to perform more specific NLP tasks such as text classification, which is exactly what we're going to do in this chapter.

> Note: When we refer to pretrained models throughout this book, we are generally referring to large, pretrained *language* models that have been trained to perform language modeling on large corpuses.

개체 지향 프로그래밍의 좋은 비유는 클래스의 상속 개념입니다. 각 동물이 클래스로 표현되는 일종의 동물원 관리 비디오 게임을 만들고 있다고 가정합니다. 동물은 체중과 키와 같은 속성뿐만 아니라 먹고 자는 것과 같은 기능도 가지고 있습니다. 이론적으로 우리는 각 동물에 대한 새 클래스를 만들고 공유 기능을 복제할 수 있지만 실제로는 일반적으로 코드에서 중복을 피하기 위해 일반적인 동물에 대한 슈퍼 클래스와 각 종에 대한 하위 클래스를 갖도록 코드를 리팩토링합니다. , 읽기 쉽게 만듭니다.

더 큰 데이터 세트에 대한 교육을 통해 모델은 기본적으로 관심 있는 작업을 더 잘 수행하는 데 사용할 수 있는 많은 양의 추가 지식을 상속합니다. 실제적인 관점에서 전이 학습은 미세 조정을 올바르게 수행할 경우 처음부터 훈련하는 것보다 종종 계산 비용이 적게 들기 때문에 더 나은 성능의 모델을 더 빠르게 얻는 데 도움이 됩니다.:[Assuming that the original dataset you're transferring *from* is much larger than the dataset you're using for fine-tuning. If your fine-tuning dataset is larger, perhaps you should be applying transfer learning the other way around! But in practice, it's very hard to natural language text datasets that are of comparable size to the ones used for pretraining.]

우리가 논의할 다른 큰 발전은 변환기(transformer)라고 하는 새로운 종류의 모델 아키텍처를 사용하는 것입니다. 훈련 변환기(transformers)는 복잡할 수 있으며 약간의 미세 조정 없이는 항상 제대로 작동하지 않습니다. 따라서 처음부터 훈련하는 대신 다른 아키텍처에 대한 사전 훈련 기술과 인기 있는 사전 훈련된 변환기(transformer)를 사용하여 추론을 수행하는 방법을 보여드리겠습니다.

이 장에서는 훈련한 모델을 다룰 것이므로 컴퓨팅 환경을 설정하는 것이 중요합니다. 이 작업을 수행하는 방법에 대한 자세한 내용은 Github 페이지를 확인하십시오. [Github page](https://github.com/nlpbook/nlpbook/)

## Training with fastai

가장 먼저 살펴볼 것은 Transfer Leaning이라는 아이디어입니다. 우리는 언어 모델을 미세 조정한 다음 감정에 따라 텍스트를 분류하는 텍스트 분류기로 변환할 것입니다. 가장 간단한 작업 구현으로 시작하여 [ULMFit](https://arxiv.org/abs/1801.06146) 기술을 사용하여 네트워크를 점진적으로 훈련할 것입니다. 이 특정 예제는 공식 fastai 문서에서 채택되었으며 라이브러리가 훌륭한 결과를 매우 빠르게 쉽게 만드는 방법을 보여주는 훌륭한 데모입니다.

여기서 사용할 데이터 세트는 IMDB 영화 리뷰 데이터 세트입니다. 그다지 재미있지는 않지만 간단하고 작기 때문에 시작할 때 우리가 원하는 것입니다.

In [ ]:
from fastai.text.all import *

### Using the high-level fastai API

`fastai`는 표준 딥 러닝 라이브러리 이상입니다. 여기에는 가능한 한 빨리 종단 간(end-to-end) 문제를 해결하는 데 도움이 되는 도구가 포함되어 있습니다. 이러한 도구 중 하나는 쉽게 다운로드할 수 있는 기본 제공 공통 데이터 집합 세트입니다.

In [ ]:
path = untar_data(URLs.IMDB)
path.ls()

In [ ]:
(path/'train').ls()

IMDB 데이터 세트의 이 특정 인스턴스는 ImageNet과 같이 구성됩니다(즉, 클래스당 하나의 디렉토리). 그래서 이 경우 긍정적인 리뷰는 `pos`에, 부정적인 리뷰는 `neg`에 저장됩니다.

`fastai`에 내장된 `TextDataLoaders.from_folder` 메서드를 사용하여 데이터 세트를 설정하고 훈련을 준비할 수 있습니다. 우리가 지정해야 하는 유일한 것은 유효성 검사 폴더의 이름인 "test"(기본 "valid"가 아님)입니다.

In [ ]:
dls = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test')

그런 다음 `show_batch` 메서드를 사용하여 데이터를 볼 수 있습니다.:

In [ ]:
dls.show_batch()

라이브러리가 분할할 모든 텍스트를 자동으로 처리한 다음 *토큰*(*tokens*)에서 다음과 같은 일부 특수 토큰을 추가하는 것을 볼 수 있습니다.
- 텍스트의 시작을 나타내는 `xxbos` 
- 다음 단어가 대문자임을 나타내는 `xxmaj` 

`fastai` 는 거의 모든 작업을 수행하기 위해 `Learner`라는 개체를 사용합니다. 한 줄의 코드에서 텍스트 분류를 위해 하나를 구성할 수 있습니다.:

In [ ]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

우리가 책의 대부분에서 극찬한(계속해서 논의할) 트랜스포머 모델 대신 [AWD LSTM](https://arxiv.org/abs/1708.02182)을 사용할 것입니다. 훈련하기가 더 쉽고 빠르기 때문에 지금은 대신 다루는 아키텍처입니다.

몇 가지 다른 세부 정보가 있습니다.: `drop_mult`는 해당 모델의 모든 드롭아웃 크기를 제어하는 매개변수이며 얼마나 잘 수행하고 있는지 추적하여 `정확도`(`accuracy`)를 사용합니다. 하지만 아직은 하이퍼파라미터에 대해 너무 걱정할 필요가 없습니다.

`Learner`가 정의되었으므로 이제 다음과 같은 놀라운 이름의 메서드를 사용하여 사전 훈련된 모델을 미세 조정할 수 있습니다.:

In [ ]:
learn.fine_tune(4, 1e-2)

In [ ]:
learn.fine_tune(4, 1e-2)

93%의 정확도가 좋아 보입니다! 하지만 실제로 얼마나 잘 작동하는지 봅시다...

In [ ]:
learn.show_results()

개별 문장에 대해 한 번에 하나씩 예측을 실행할 수도 있습니다.:

In [ ]:
learn.predict("That movie was wicked cool!")

여기에서 모델이 리뷰를 긍정적으로 간주한 것을 볼 수 있습니다. 결과의 두 번째 부분은 데이터 어휘의 "pos" 인덱스이고 마지막 부분은 각 클래스에 할당된 확률입니다("pos"의 경우 99.1%, "neg"의 경우 0.9%).

#### Building a Dataset with fastai's DataBlock API

또한 `DataLoaders`에서 데이터를 가져와서 `fastai` 데이터 블록 API를 사용할 수 있습니다. 이것은 좀 더 고급이므로 아직 `fastai`에 익숙하지 않다면 이 부분을 건너뛰어도 됩니다. 이 접근 방식은 결국 동일한 결과를 제공합니다.

데이터 블록은 fastai 라이브러리에 많은 정보를 제공하여 구축됩니다.:

- `blocks`라는 인수를 통해 사용된 유형: 여기에는 이미지와 범주가 있으므로 `TextBlock` 및 `CategoryBlock`을 전달합니다. 라이브러리에 텍스트가 폴더의 파일임을 알리기 위해 `from_folder` 클래스 메서드를 사용합니다.
- 원본 항목을 가져오는 방법, 여기서는 `get_text_files` 함수입니다.
- 여기에서 상위 폴더와 함께 해당 항목에 레이블을 지정하는 방법.
- 해당 항목을 분할하는 방법, 여기에서 grandparent 폴더와 함께.

In [ ]:
imdb = DataBlock(blocks=(TextBlock.from_folder(path), CategoryBlock),
                 get_items=get_text_files,
                 get_y=parent_label,
                 splitter=GrandparentSplitter(valid_name='test'))

이것은 데이터를 어셈블하는 방법에 대한 청사진만 제공합니다. 실제로 생성하려면 `dataloaders` 메서드를 사용해야 합니다.:

In [ ]:
dls = imdb.dataloaders(path)

### ULMFiT for Transfer Learning

이전 섹션에서 사용한 사전 훈련된 모델을 언어 모델이라고 합니다. 이전에 모든 단어를 읽은 후 일련의 Wikipedia 기사에서 다음 단어를 추측하도록 훈련되었습니다. 이 언어 모델을 영화 리뷰 분류기로 직접 미세 조정하여 훌륭한 결과를 얻었지만 한 단계만 추가하면 더 잘할 수 있습니다.

Wikipedia 영어는 IMDb 영어와 약간 다릅니다. 따라서 분류기로 직접 이동하는 대신 미리 훈련된 언어 모델을 IMDb 데이터 세트로 미세 조정한 다음 위키백과 언어 모델 대신 *그*(*that*)를 분류기의 기반으로 사용할 수 있습니다.

이는 직관적으로 이해가 됩니다. 글을 읽을 수 있는 인간으로서 영화 리뷰가 일반적으로 어떤 것인지에 대한 맥락을 이해한다면 아마도 영화 리뷰를 분류하는 데 더 나은 작업을 할 수 있을 것입니다. SAT를 보기 며칠 전에 지문을 미리 읽는 것과 같습니다. 여기서만 우리는 친구이기 때문에 언어 모델을 부정행위로 부르지 않을 것입니다. 제발 날 먹지마, 미래의 로봇 대군주.].

그러나 그 외에도 이것이 유용한 또 다른 매우 중요한 이유는 *라벨이 지정된*(*labelled*) 데이터보다 더 많은 데이터가 있는 경우가 많기 때문입니다. 레이블 지정은 비용이 많이 들고 일반적으로 사람의 시간과 노력이 필요하므로 문서 태깅과 같이 작은 하위 집합만 사용되는 대규모 텍스트 레코드 데이터베이스를 보유하는 것은 드문 일이 아닙니다. 그러나 이 미세 조정 접근 방식을 사용하면 분류자를 훈련시키기 전에 레이블이 지정되지 않은 데이터를 사용하여 *언어 모델*(*language model*)을 미세 조정할 수 있습니다.

결함이 있는 유추를 끌어올 위험을 무릅쓰고 이것은 거의 수년간의 이전 SAT 구절에 접근하는 것과 같습니다. 그들 중 누구도 시험에 *정확히*(*exactly*) 나타나지는 않지만, 연습을 해보면 SAT가 어떤 것인지 이해하는 데 도움이 될 것입니다.

이 접근 방식은 2018년 Jeremy Howard와 Sebastian Ruder가 소개한 ULMFiT라고 합니다.:[Who also happends to be the creator of fastai!] 이것 관련 과정은 [[ulmfit]]에 요약되어 있습니다. 

![ULMFit](images/ulmfit.png)

화살표와 원은 모든 것을 훨씬 더 간단하게 만듭니다. 그렇죠?

사전 훈련된 Wikipedia 언어 모델이 이미 있으므로 [[ulmfit]]에서 piple의 2단계인 IMDB 언어 모델 미세 조정부터 시작할 수 있습니다.

### Fine-tuning a language model on IMDb

언어 모델링에 적합한 `DataLoaders`에서 매우 쉽게 텍스트를 가져올 수 있습니다.:

In [ ]:
dls_lm = TextDataLoaders.from_folder(path, is_lm=True, valid_pct=0.1)

`valid_pct`에 대해 무언가를 전달해야 합니다. 그렇지 않으면 이 메서드는 최상위 폴더 이름을 사용하여 데이터를 분할하려고 시도합니다. `valid_pct=0.1`을 전달함으로써 검증 세트에 대한 리뷰의 10%를 임의로 가져오도록 지시합니다.

`show_batch`를 사용하여 데이터를 볼 수 있습니다. 여기서 작업은 다음 단어를 추측하는 것이므로 대상이 모두 오른쪽으로 한 단어 이동한 것을 볼 수 있습니다.

In [ ]:
dls_lm.show_batch(max_n=5)

그런 다음 이전과 같이 `AWD_LSTM` 아키텍처를 사용하여 `Learner`를 직접 가져올 수 있는 편리한 방법이 있습니다. 정확성과 당혹감을 지표로 사용하고(나중은 손실의 지수임) 기본 가중치 감쇠를 0.1로 설정합니다. `to_fp16`은 `Learner`를 혼합 정밀도로 설정하여 Tensor 코어가 있는 GPU에서 훈련 속도를 높이는 데 도움이 됩니다.

In [ ]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, metrics=[accuracy, Perplexity()],
    path=path, wd=0.1).to_fp16()

기본적으로 사전 훈련된 '학습자'(`Learner`)는 동결 상태입니다. 즉, 모델의 머리만 훈련하고 신체는 동결된 상태를 유지합니다. 여기에서 fine_tune 방법 뒤에 무엇이 있는지 보여주고 fit_one_cycle 방법을 사용하여 모델을 맞춥니다.:

In [ ]:
learn.fit_one_cycle(1, 1e-2)

이 모델은 학습하는 데 시간이 걸리므로 중간 결과 저장에 대해 이야기할 수 있는 좋은 기회입니다.

다음과 같이 모델의 상태를 쉽게 저장할 수 있습니다.:

In [ ]:
learn.save('1epoch')

`learn.path/models/`에 "1epoch.pth"라는 파일이 생성됩니다. 동일한 방식으로 '학습자'를 생성한 후 다른 컴퓨터에 모델을 로드하거나 나중에 교육을 다시 시작하려는 경우 다음을 사용하여 이 파일의 콘텐츠를 로드할 수 있습니다.:

In [ ]:
learn = learn.load('1epoch')

고정 해제 후 모델을 미세 조정할 수 있습니다.:

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-3)

이 작업이 완료되면 활성화를 어휘에서 각 토큰을 선택할 확률로 변환하는 최종 레이어를 제외한 모든 모델을 저장합니다. 최종 레이어를 포함하지 않는 모델을 *인코더*(*encoder*)라고 합니다. `save_encoder`로 저장할 수 있습니다:

In [ ]:
learn.save_encoder('finetuned')

.Who's That Pokémon?
> Tip: The encoder is the model not including the task-specific final layer(s). It means much the same thing as *body* when applied to vision CNNs, but tends to be more used for NLP and generative models.

이를 사용하여 리뷰에서 분류기를 미세 조정하기 전에 모델을 사용하여 무작위 리뷰를 생성할 수 있습니다. 문장의 다음 단어가 무엇인지 추측하도록 훈련되었기 때문에 새로운 리뷰를 작성하는 데 사용할 수 있습니다.:

In [ ]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [ ]:
print("\n".join(preds))

영화 리뷰에서 언어 모델이 미세 조정되었으므로 이제 영화 리뷰를 *분류*(*classify*)하도록 수정할 수 있습니다. 아이디어는 이 시점에서 모델이 다음 단어를 예측할 수 있을 만큼 "충분히 똑똑"하다면 단순한 양성/음성 분류가 *반드시*(*must*) 가능해야 한다는 것입니다.

### Training a text classifier

이전과 거의 동일하게 텍스트 분류를 위해 데이터를 수집할 수 있습니다.:

In [ ]:
dls_clas = TextDataLoaders.from_folder(
    untar_data(URLs.IMDB), valid='test',
    text_vocab=dls_lm.vocab)

주요 차이점은 언어 모델을 미세 조정할 때와 똑같은 어휘를 사용해야 한다는 것입니다. 그렇지 않으면 학습된 가중치가 의미가 없습니다. 우리는 `text_vocab`로 그 어휘를 전달합니다.

그런 다음 이전과 같이 텍스트 분류기를 정의할 수 있습니다.

In [ ]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

차이점은 학습하기 전에 이전 인코더를 로드한다는 것입니다.:

In [ ]:
learn = learn.load_encoder('finetuned')

마지막 단계는 차별적인 학습 속도와 *점진적인 동결 해제*(*gradual unfreezing*)로 훈련하는 것입니다. 컴퓨터 비전에서 우리는 종종 모델을 한 번에 모두 고정 해제하지만 NLP 분류기의 경우 한 번에 몇 개의 레이어를 고정 해제하는 것이 큰 차이를 만든다는 것을 알게 되었습니다.

In [ ]:
learn.fit_one_cycle(1, 2e-2)

단 한 에포크(epoch)에서 우리는 첫 번째 섹션의 훈련과 동일한 결과를 얻습니다. 나쁘지 않습니다! `-2`를 `freeze_to`에 전달하여 마지막 두 매개변수 그룹을 제외한 모든 그룹을 동결할 수 있습니다.:

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

그런 다음 조금 더 고정을 해제하고 훈련을 계속할 수 있습니다.:

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

마지막으로 전체 모델을 고정 해제하고 모든 계층을 훈련시켜 정확도를 최종적으로 높일 수 있습니다.

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

이제 리뷰의 원시 텍스트 콘텐츠만을 기반으로 영화 리뷰에 긍정적인 감정이 있는지 부정적인 감정이 있는지 정확하게 예측할 수 있는 텍스트 분류 모델이 있습니다. `fastai` API를 이해하면 이제 선택한 데이터 세트에 고유한 텍스트 분류자를 구현할 수 있습니다.

IMDB 자체는 상당히 단순했지만 오늘날 많은 NLP 문제는 텍스트 분류 문제로 공식화될 수 있습니다. 텍스트 분류로 수행할 수 있는 몇 가지 작업은 다음과 같습니다.

- 일부 소스 코드의 프로그래밍 언어 예측
- 간단한 이메일 스팸 분류기 구축
- 온라인 채팅 또는 포럼을 위한 자동화된 콘텐츠 조정 봇의 기능 개선
- 언어 각주를 기준으로 문서 분류:[To do this well, you need a powerful tokenizer that can recognize text encoding in many languages]

텍스트 분류의 가장 좋은 부분 중 하나는 최적화를 위한 단순하고 해석 가능한 단일 메트릭인 정확도가 있다는 것입니다. 따라서 이러한 작업을 해결할 수 있을 뿐만 아니라 많은 사람들에게 익숙한 통계를 사용하여 얼마나 잘하고 있는지도 알 수 있습니다.

우리가 지금 구축한 IMDB 모델은 훌륭한 작업을 수행하지만 그다지 인상적이지는 않을 것입니다. 우리는 Dinosaurs의 여명기부터 꽤 잘 작동하는 스팸 분류기를 가지고 있었으므로 텍스트에 대한 이진 예측은 [[ch01]]에서 우리가 당신에게 판매한 영광스러운 미래와 연관시킬 수 있는 것이 아닙니다. 그러나 언어 모델에 대한 이 아이디어는 매우 강력하여 오늘날 NLP의 포스터 자식이 되었습니다.

이를 설명하기 위해 추가 교육이나 미세 조정 없이 자체적으로 언어 모델을 제공하여 근육을 유연하게 변경해 보겠습니다.

## Inference with Huggingface

이제 우리는 언어 모델을 훈련하는 방법을 알았으므로 많은 데이터에 대해 매우 큰 것을 개념적으로 훈련하고 매우 정확한 발음 텍스트를 생성하도록 할 수 있습니다. 여기서는 위에서 사용한 것과 유사한 절차를 사용하여 훈련된 언어 모델에서 예측 샘플을 얻기 위해 huggingface 라이브러리를 사용할 것입니다.

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pre-trained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Encode a text inputs
text = "With great power comes great "
indexed_tokens = tokenizer.encode(text)

# Convert indexed tokens in a PyTorch tensor
tokens_tensor = torch.tensor([indexed_tokens])

위의 코드 스니펫은 토크나이저를 초기화합니다. 토크나이저는 문자열을 입력으로 사용하고 모델이 해석하기 더 쉬운 숫자 배열을 반환하는 함수입니다. 다음 장에서 토크나이저에 대해 더 자세히 다루겠지만 지금은 모델이 보는 것을 빠르게 살펴보고 싶다면 `tokens_tensor`를 인쇄해 보십시오.

In [ ]:
print(tokens_tensor)

이제 실제 추론을 해봅시다. 다시 말하지만 놀라운 huggingface 변환기 라이브러리 덕분에 몇 줄의 코드에 불과합니다.

In [ ]:
# Load pre-trained model (weights)
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Set the model in evaluation mode to deactivate the DropOut modules
# This is IMPORTANT to have reproducible results during evaluation!
model.eval()

# Predict all tokens
with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0]

# get the predicted next sub-word
predicted_index = torch.argmax(predictions[0, -1, :]).item()
predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])
print(predicted_text)

Nice! 우리가 방금 실행한 것이 무엇이든 단 몇 줄의 코드로 벤 삼촌의 지혜를 재현할 수 있었던 것 같습니다!footnote[A character from the Spider-Man comics book series, who once said "with great power comes great responsibility," just like our language model did!]

분명히 말씀드리자면 이것은 단순한 조회, 데이터베이스 검색 또는 이와 유사한 것이 아닙니다. 이것은 인터넷에서 많은 양의 텍스트를 읽은 후 얻은 "지식"을 기반으로 문장을 완성할 수 있는 실제 최첨단 신경망이었습니다. 꽤 멋지죠?

그러나 맥락이 없으면 이것은 모두 당신이 문장을 던지는 블랙 박스에 불과합니다. 이제 방금 실행한 블록의 각 줄 코드를 분석하여 무슨 일이 일어나고 있는지 제대로 파악해 보겠습니다.

### Loading Models

먼저 사전 학습된 모델을 로드합니다. 이것은 전이 학습에서 가장 중요한 단일 단계입니다. 인터넷 어딘가에서 예측하는 데 사용할 모델을 다운로드하고 올바른 형식으로 코드의 개체에 로드합니다. 이 모든 기능은 고맙게도 다음 한 줄의 코드에 포함되어 있습니다.:

In [ ]:
model = GPT2LMHeadModel.from_pretrained('gpt2')

대부분의 딥 러닝 라이브러리는 이 모델 로딩 기능을 간단한 함수로 깔끔하게 패키징합니다. 그것은 당신이 걱정해야 할 마지막 것입니다.

여기에서 로드하는 특정 모델은 이 단계에서 중요하지 않지만 "GPT2"라고 합니다. 이 모델은 처음 나왔을 때 정말 혁신적이었고 기본적으로 인터넷을 망쳤습니다. Ajay가 2019년에 작성한[an article](https://blog.floydhub.com/gpt2/) 에서 자세한 내용을 읽을 수 있지만 이 책의 [[ch09]]에서도 이야기하겠습니다.

.Loading Models
> **참고:** 작업이나 도메인에 관계없이 모델을 +model+라는 변수에 로드하는 것은 딥 러닝에서 매우 흔한 일이므로 온라인에서 노트북이나 코드 샘플을 탐색할 때 이 점을 염두에 두십시오.

다음으로 이 작은 코드 라인을 실행합니다. 이 코드 라인은 우리가 지금 훈련하고 있지 않고 대신 예측(즉, 추론 수행)을 할 것임을 모델에 알려줍니다. 이 linefootmote를 호출할 때 `model` 개체에서 내부적으로 변경되는 몇 가지 사항이 있습니다.:[주로 교육 중에만 유용한 DropOut 및 BatchNorm 레이어를 비활성화하여] '모델'에서 예측을 생성할 수 있습니다. 다시 말하지만 이것은 우리가 지금 하고 있는 일에서 가장 중요한 라인은 아니지만 예측을 생성하고 싶을 때마다 이 함수를 호출해야 합니다. 노트북에서 이 줄을 실행하면 모델의 모든 레이어가 표준 PyTorch 형식으로 인쇄되므로 궁금한 점이 있으면 스크롤하여 살펴보십시오.

In [ ]:
model.eval()

가중치를 다운로드하고 메모리에 로드한 모델과 평가 모드로 설정한 `모델` 개체를 사용하여 간단한 각주에서 일부 출력을 생성할 시간입니다.[좋아요, 아마도 이 문구는 GPT-2에 특별히 적용되지 않을 수도 있지만, 우리 모두가 오늘날보다 300배 더 빠른 컴퓨터를 가지고 있다면 이 형용사는 정확할 것입니다.], 텍스트 생성 기계를 의미합니다.

### Generating Predictions

다음 세 줄은 하나의 블록으로 작동하므로 함께 그룹화할 것입니다.

In [ ]:
with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0]

첫 번째 줄인 `with torch.no_grad():`는 PyTorch에게 `torch.no_grad()` 컨텍스트의 해당 들여쓰기 블록에 있는 줄을 실행하도록 지시합니다. 모델. 역전파에 익숙하지 않거나 처음에 그래디언트가 계산되는 이유가 완전히 명확하지 않은 경우 소개에 있는 리소스를 참조하세요. 엄밀히 말하면 그래디언트 계산을 해제할 필요는 없지만 이렇게 하면 시간, 메모리 및 계산이 절약되고 추론이 더 빠르게 실행됩니다.

`torch.no_grad()` 컨텍스트에서 순방향 패스를 실행합니다. 항상 그렇듯이 PyTorch는 이것을 매우 간단하게 만듭니다. 위에서 준비한 `tokens_tensor`를 입력으로 하여 `model`을 함수로 호출하기만 하면 됩니다.

하지만 잠깐, +model+이 위에서 로드한 사전 훈련된 가중치를 가진 객체가 아니었나요? 기능도 어떻습니까?

.Python Dunder Methods
****

In Python, you can actually do this! You have to define `__call__` method in your class, which is a special function called a dunder method. Python has a lot of these cool dunder functions, some of which you've likely encounter before, like `__init__`, which let's you set up a constructor for your class, and `__len__`, which let's you define a "length" property for your objects which you can access via the `len()` function. Python dunder methods allow you to define a lot of cool functionality for your custom classes, such as addition, equality, and more.

If you define a function called `__call__()` in your Python class, you can then treat instances of your class as functions, and the `__call__()` function will be invoked everytime you do so. We'll soon talk about PyTorch `nn.Module` objects, which are the building blocks for neural nets. The `nn.Module` class implements the `__call__` function by default. Therefore, every PyTorch model (and submodule) can also be called as a function, which can make your code very neat and tidy. This is why we can both define the +model+ variable and call it as we would for a function at the same time.

If you're interested in learning more about python dunder methods, check out this [tutorial](https://rszalski.github.io/magicmethods/) or read more online (there are plenty of great resources one search away).

****

일반적으로 `model(input_tensor)`를 호출하면 예측과 함께 `torch.tensor` 객체가 반환됩니다. 그러나이 경우 huggingface 라이브러리는 실제로 다른 많은 항목도 제공합니다. 이 경우 `model(tokens_tensor)`는 첫 번째 요소가 예측 텐서인 튜플을 반환합니다. 몇 가지 길이와 모양을 확인하여 이 모든 것을 빠르게 확인합시다.

In [ ]:
outputs[0].shape

이는 huggingface 변환기 문서에 따르면 예측 텐서가 `(batch_size, sequence_length, config.vocab_size)` 모양을 가져야 하기 때문에 확인됩니다. 여기서는 한 문장만 전달하므로 배치 크기는 1입니다. 시퀀스 길이는 5여야 합니다. 문자열에 5개 단어(공백으로 구분된 하위 문자열)가 있는 입력 문장을 정의하는 줄을 보면 의미가 있습니다.:

```
text = "고생 끝에 낙이 온다"("With great power comes great")
```

어휘 크기에 대한 50257 값이 정확해 보이지만 이 값은 이 모델에 대한 문서를 검토하여 항상 다시 확인할 수 있는 것입니다.


> **팁:** `torch.tensors`의 크기, 모양 및 크기를 확인하는 이 기술이 얼마나 중요한지 아무리 강조해도 지나치지 않습니다. 모델을 디버깅하는 가장 효과적인 방법 중 하나입니다. 바라건대, 더 복잡한 모델을 훈련하고 자신만의 아키텍처를 처음부터 구축하기 시작하면 이것이 자연스럽게 이루어질 것입니다. 하지만 그때까지는 항상 `.size`로 크기를 확인하고 모델에서 진행되는 작업을 통해 추론하는 것을 잊지 마십시오.

`outputs[0]`이 우리가 원하는 것 같기 때문에 `predictions` 변수에 할당하겠습니다. 이들을 함께 넣고 `torch.no_grad()` 컨텍스트에 래핑하면 위에 있는 코드의 작은 블록을 얻을 수 있습니다.

In [ ]:
with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0]

`예측`(`predictions`)은 각 단어의 확률을 설명하는 값이 있는 `torch.tensor`입니다. 이 `torch.tensor`의 차원 중 하나는 어휘의 크기(즉, 모델이 예측할 수 있는 가능한 단어의 수)라는 점을 기억하세요. 지금 우리가 원하는 것은 문장에서 다음에 올 가능성이 가장 높은 단어입니다. 배열에서 가장 큰 값의 인덱스를 가져오는 `argmax` 함수를 사용하여 이를 가져옵니다.

In [ ]:
predicted_index = torch.argmax(predictions[0, -1, :]).item()

우리가 정확히 무엇을 하고 있는지 확실히 하기 위해 `예측`(`predictions`)을 인덱싱하는 방법도 빠르게 분석해 보겠습니다. 3차원 텐서이므로 3개의 인덱스를 지정합니다. 배치 차원을 따라 첫 번째는 `0`입니다. 배치 예측을 실행하지 않기 때문에 이 축에는 요소가 하나만 있으므로 선택합니다. 시퀀스 길이 차원을 따라 마지막 요소를 선택합니다. 전달한 문장의 마지막 단어를 예측하고 싶기 때문입니다. 마지막 인덱스는 `:`이며, 이는 모든 것을 잡고 싶다는 의미입니다. 가장 가능성이 높은 항목을 계산하려면 어휘 차원을 따라 모든 요소가 필요합니다.

마지막으로 +tokenizer.decode()+ 기능을 사용하여 단어로 얻은 인덱스를 디코딩합니다. 이것은 단순한 조회일 뿐입니다.

In [ ]:
predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])
print(predicted_text)

그리고 우리는 그것을 가지고 있습니다! 단 몇 줄의 코드로 지혜를 재현합니다.

## Conclusion

[[ch08]]에서 우리는 이러한 아이디어를 결합하여 트랜스포머와 전이 학습을 함께 활용하는 기술을 개발하여 방금 시연한 이러한 작업과 더 많은 작업을 해결할 수 있는 매우 강력한 모델 세트를 만들 것입니다.
    
이 장에는 아직 설명하지 않은 내용이 많이 있습니다. 우리는 모델이 정확히 무엇인지/하는 일, 토크나이저가 코드에서 어떻게 구현되는지, 아마도 가장 중요한 것은 전이 학습을 위해 사전 훈련된 모델을 사용하는 방법과 같은 많은 세부 사항을 의도적으로 생략했습니다.

하지만 걱정하지 마세요. 우리는 결국 그 모든 것을 알게 될 것입니다. 이 장의 목표는 실시간으로 코드를 실행하고 결과를 확인하여 NLP 파이프라인의 중요한 구성 요소 중 일부를 이해하는 데 도움을 주는 것입니다. 지금까지 자료에 대한 이해도를 테스트하려면 다른 언어 모델을 사용하고 프롬프트를 교체한 다음 모델이 인기 있는 인용구, 구 또는 숙어를 예측하도록 할 수 있는지 확인하십시오. 이렇게 하려면 토크나이저도 교체해야 할 수 있습니다.
    
이러한 작업을 수행할 수 있으면 다음 장으로 넘어갈 준비가 된 것입니다. 이 장에서는 오늘날 가장 인기 있는 NLP 응용 프로그램을 공식적으로 소개하고 몇 가지를 함께 구축합니다.